In [ ]:
"""
people with higher incomes (>50k) are more likely to accept the special promotion offer 10%
for people with lower lower income (<=50k) only 5% accept the special promotion offer

people with high income, on average, gives us 980 euro in profit
while people with lower income costs us 310 euro on average

sending a special promotion offer costs 10 euro per package

two .xlsx files are provided, "01_existing-customers.xlsx" which contains whether or not the person exceeds 50k income 
and "01_potential-customers.xlsx" which does not contain data about income
both of the files contains the following attributes:
1. age
2. workclass
3. education: highest level of education achieved by an individual
4. education-num: highest level of educationa chieved in numerical form (an integer greater than 0)
5. marital-status: marital status of an individual
6. occupation: general type of occupation of an individual
7. relationship: representa what this individual is relative to others
8. race
9. sex
10. capital-gain: capital gains for an individual (integer greater than or equal to 0)
11. capital-loss: capital losses for an individual (integer greater than or equal to 0)
12. hours-per-week: the hours an individual has reported to work per week continuous
13. native-country: country of origin for an individual
lastly, "01_existing-customers.xlsx" contains the extra attribute:
14. the label: whether or not an individual makes more than 50k a year

Solve this problem (provide the list of people to send the promotion to) and give an estimate of the profit you expect when sending the promotion to the people you selected.
The goal is to maximize the revenue

make use of the library: sklearn
first use decision trees to solve this problem
"""
import pandas as pd

# read the data
# assign column names to dataset
column_names = ["age", "workclass",	"education",	"education-num",	"marital-status", "occupation",
                "relationship",	"race",	"sex",	"capital-gain",	"capital-loss",	"hours-per-week",	"native-country"]

existing_customers = pd.read_excel("01_existing-customers.xlsx", usecols=column_names)
existing_class = pd.read_excel("01_existing-customers.xlsx", usecols=["class"])
# remove the last column name "class" from the list as it does not exist in the potential_customers dataset
potential_customers = pd.read_excel("01_potential-customers.xlsx", usecols=column_names)


In [50]:
# check the data
print("Existing Customers:")
existing_customers

Existing Customers:


,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


In [51]:
print("\nPotential Customers:")
potential_customers


Potential Customers:


,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
16277,64,NaN,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States
16278,38,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
16279,44,Private,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


# Convert data into numerical data

In [54]:
# Turn the categorical data into numerical data
from sklearn import preprocessing 
# categorical_features = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
categorical_features = column_names
encoder = preprocessing.OrdinalEncoder()

encoder.fit(existing_customers)
transformed_existing_customers = encoder.transform(existing_customers)

encoder.fit(existing_class)
transformed_existing_class = encoder.transform(existing_class)

encoder.fit(potential_customers)
transformed_potential_customers = encoder.fit_transform(potential_customers)

In [41]:
pd.DataFrame(transformed_existing_customers)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,22.0,6.0,9.0,12.0,4.0,0.0,1.0,4.0,1.0,25.0,0.0,39.0,38.0
1,33.0,5.0,9.0,12.0,2.0,3.0,0.0,4.0,1.0,0.0,0.0,12.0,38.0
2,21.0,3.0,11.0,8.0,0.0,5.0,1.0,4.0,1.0,0.0,0.0,39.0,38.0
3,36.0,3.0,1.0,6.0,2.0,5.0,0.0,2.0,1.0,0.0,0.0,39.0,38.0
4,11.0,3.0,9.0,12.0,2.0,9.0,5.0,2.0,0.0,0.0,0.0,39.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,10.0,3.0,7.0,11.0,2.0,12.0,5.0,4.0,0.0,0.0,0.0,37.0,38.0
32557,23.0,3.0,11.0,8.0,2.0,6.0,0.0,4.0,1.0,0.0,0.0,39.0,38.0
32558,41.0,3.0,11.0,8.0,6.0,0.0,4.0,4.0,0.0,0.0,0.0,39.0,38.0
32559,5.0,3.0,11.0,8.0,4.0,0.0,3.0,4.0,1.0,0.0,0.0,19.0,38.0


In [42]:
pd.DataFrame(transformed_potential_customers)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,8.0,3.0,1.0,6.0,4.0,6.0,3.0,2.0,1.0,0.0,0.0,39.0,37.0
1,21.0,3.0,11.0,8.0,2.0,4.0,0.0,4.0,1.0,0.0,0.0,49.0,37.0
2,11.0,1.0,7.0,11.0,2.0,10.0,0.0,4.0,1.0,0.0,0.0,39.0,37.0
3,27.0,3.0,15.0,9.0,2.0,6.0,0.0,2.0,1.0,90.0,0.0,39.0,37.0
4,1.0,NaN,15.0,9.0,4.0,NaN,3.0,4.0,0.0,0.0,0.0,29.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,22.0,3.0,9.0,12.0,0.0,9.0,1.0,4.0,0.0,0.0,0.0,35.0,37.0
16277,47.0,NaN,11.0,8.0,6.0,NaN,2.0,2.0,1.0,0.0,0.0,39.0,37.0
16278,21.0,3.0,9.0,12.0,2.0,9.0,0.0,4.0,1.0,0.0,0.0,49.0,37.0
16279,27.0,3.0,9.0,12.0,0.0,0.0,3.0,1.0,1.0,75.0,0.0,39.0,37.0


In [ ]:
# split into X and y, 
# where X is the feature matrix
# and y is the column we want to predict


# split into training and test set

# Fill in the empty cells with Multivariate imputation

In [55]:
# notice we have missing values in the dataset
# to solve this we can make use of multivariate imputation from sklearn
# https://scikit-learn.org/stable/modules/impute.html
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(transformed_existing_customers)
transformed_existing_customers = pd.DataFrame(imp.transform(transformed_existing_customers), columns=column_names)

imp.fit(transformed_existing_class)
transformed_existing_class = pd.DataFrame(imp.transform(transformed_existing_class))

imp.fit(transformed_potential_customers)
transformed_potential_customers = pd.DataFrame(imp.transform(transformed_potential_customers), columns=column_names)


In [46]:
transformed_existing_customers

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,22.0,6.0,9.0,12.0,4.0,0.0,1.0,4.0,1.0,25.0,0.0,39.0,38.0
1,33.0,5.0,9.0,12.0,2.0,3.0,0.0,4.0,1.0,0.0,0.0,12.0,38.0
2,21.0,3.0,11.0,8.0,0.0,5.0,1.0,4.0,1.0,0.0,0.0,39.0,38.0
3,36.0,3.0,1.0,6.0,2.0,5.0,0.0,2.0,1.0,0.0,0.0,39.0,38.0
4,11.0,3.0,9.0,12.0,2.0,9.0,5.0,2.0,0.0,0.0,0.0,39.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,10.0,3.0,7.0,11.0,2.0,12.0,5.0,4.0,0.0,0.0,0.0,37.0,38.0
32557,23.0,3.0,11.0,8.0,2.0,6.0,0.0,4.0,1.0,0.0,0.0,39.0,38.0
32558,41.0,3.0,11.0,8.0,6.0,0.0,4.0,4.0,0.0,0.0,0.0,39.0,38.0
32559,5.0,3.0,11.0,8.0,4.0,0.0,3.0,4.0,1.0,0.0,0.0,19.0,38.0


In [47]:
transformed_potential_customers

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,8.0,3.000000,1.0,6.0,4.0,6.000000,3.0,2.0,1.0,0.0,0.0,39.0,37.0
1,21.0,3.000000,11.0,8.0,2.0,4.000000,0.0,4.0,1.0,0.0,0.0,49.0,37.0
2,11.0,1.000000,7.0,11.0,2.0,10.000000,0.0,4.0,1.0,0.0,0.0,39.0,37.0
3,27.0,3.000000,15.0,9.0,2.0,6.000000,0.0,2.0,1.0,90.0,0.0,39.0,37.0
4,1.0,2.974661,15.0,9.0,4.0,5.575642,3.0,4.0,0.0,0.0,0.0,29.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,22.0,3.000000,9.0,12.0,0.0,9.000000,1.0,4.0,0.0,0.0,0.0,35.0,37.0
16277,47.0,3.126844,11.0,8.0,6.0,5.796052,2.0,2.0,1.0,0.0,0.0,39.0,37.0
16278,21.0,3.000000,9.0,12.0,2.0,9.000000,0.0,4.0,1.0,0.0,0.0,49.0,37.0
16279,27.0,3.000000,9.0,12.0,0.0,0.000000,3.0,1.0,1.0,75.0,0.0,39.0,37.0


# Build machine learning models

In [58]:
# split into X and y
X = transformed_existing_customers  # feature matrix
y = transformed_existing_class  # column we want to predict

# split into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train = np.ravel(y_train)


### K Nearest Neighbor

In [92]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=8)
model.fit(X_train, y_train)
model.score(X_test, y_test)

c:\Users\yanok\Desktop\Data_Mining\.venv\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.833256563795486

### Decision tree

In [75]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=15)
model.fit(X_train, y_train)
model.score(X_test, y_test)


0.8447719944725933

### Naive Bayes

In [94]:
# Naive bayes
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train, y_train)
model.score(X_test, y_test)

c:\Users\yanok\Desktop\Data_Mining\.venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8157531091662829

# Combining classifiers

### Bagging

In [97]:
# Bagging: Decision Tree
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=15), n_estimators=10)
model.fit(X_train, y_train)
model.score(X_test, y_test)


c:\Users\yanok\Desktop\Data_Mining\.venv\lib\site-packages\sklearn\ensemble\_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\yanok\Desktop\Data_Mining\.venv\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.8567480423767849

In [98]:
# Bagging: KNN

model = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=8), n_estimators=10)
model.fit(X_train, y_train)
model.score(X_test, y_test)

c:\Users\yanok\Desktop\Data_Mining\.venv\lib\site-packages\sklearn\ensemble\_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\yanok\Desktop\Data_Mining\.venv\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.8323353293413174

In [99]:
# Bagging: Naive Bayes

model = BaggingClassifier(base_estimator=GaussianNB(), n_estimators=10)
model.fit(X_train, y_train)
model.score(X_test, y_test)

c:\Users\yanok\Desktop\Data_Mining\.venv\lib\site-packages\sklearn\ensemble\_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\yanok\Desktop\Data_Mining\.venv\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.8157531091662829

### Random Forest

In [101]:
# Random forest classifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=15)
model.fit(X_train, y_train)
model.score(X_test, y_test)

C:\Users\yanok\AppData\Local\Temp\ipykernel_23072\1175760817.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


0.8593582066635959

### Conclusion
It is clear that Random Forest gave the best result, so we try to find the most optimal depth

In [107]:
from sklearn.model_selection import GridSearchCV
rfc=RandomForestClassifier(random_state=0)
param_grid = {
    'n_estimators': [200, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [13, 15, 17, 19],
    'criterion' :['gini', 'entropy']
} # https://www.kaggle.com/code/sociopath00/random-forest-using-gridsearchcv/notebook
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, verbose=10)
CV_rfc.fit(X_train, y_train)
CV_rfc.best_params_

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5; 1/32] START criterion=gini, max_depth=13, max_features=sqrt, n_estimators=200
[CV 1/5; 1/32] END criterion=gini, max_depth=13, max_features=sqrt, n_estimators=200;, score=0.866 total time=   2.1s
[CV 2/5; 1/32] START criterion=gini, max_depth=13, max_features=sqrt, n_estimators=200
[CV 2/5; 1/32] END criterion=gini, max_depth=13, max_features=sqrt, n_estimators=200;, score=0.868 total time=   2.1s
[CV 3/5; 1/32] START criterion=gini, max_depth=13, max_features=sqrt, n_estimators=200
[CV 3/5; 1/32] END criterion=gini, max_depth=13, max_features=sqrt, n_estimators=200;, score=0.861 total time=   2.0s
[CV 4/5; 1/32] START criterion=gini, max_depth=13, max_features=sqrt, n_estimators=200
[CV 4/5; 1/32] END criterion=gini, max_depth=13, max_features=sqrt, n_estimators=200;, score=0.861 total time=   2.1s
[CV 5/5; 1/32] START criterion=gini, max_depth=13, max_features=sqrt, n_estimators=200
[CV 5/5; 1/32] END criterion=gi

{'criterion': 'gini',
 'max_depth': 15,
 'max_features': 'sqrt',
 'n_estimators': 500}

In [112]:
model = RandomForestClassifier(max_depth=15, max_features='sqrt', n_estimators=500)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.8635037617073545